In [125]:
import pandas as pd

In [126]:
df = pd.read_excel('data/arquivo_anonimizado.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)

In [127]:
df.head(5)

,matriculaDRE,periodoIngressoUFRJ,cursoIngressoUFRJ,cursoAtual,periodoIngressoCursoAtual,situacaoMatriculaAtual,sexo,disciplinasCursadas,crPorPeriodo,craPorPeriodo,periodosIntegralizados,cargaHorariaAcumulada,CRA,periodosTrancados,periodosCancelados,periodosCRMenor3,reprovacoes
0,0,2000/1,Bacharelado em Informática,Bacharelado em Ciência da Computação,2000/1,Cancelada por conclusão de curso,M,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - 7.6\n2000/2 - 7.1\n2001/1 - 5.9\n2001...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,13,3105,6.6,NaN,NaN,2005/2,2001/1 - MAB243 Organizacao de Dados I - 020 -...
1,1,2000/1,Bacharelado em Informática,Bacharelado em Ciência da Computação,2000/1,Cancelada por conclusão de curso,M,2000/1 - MAA123 Algebra para Informatica - 062...,2000/1 - 7.6\n2000/2 - 6.5\n2001/1 - 5.1\n2001...,2000/1 - 7.6\n2000/2 - 7\n2001/1 - 6.4\n2001/2...,10,3330,6.4,NaN,2005/1,NaN,2001/1 - FIT121 Física II - 041 - Reprovado me...
2,2,2000/1,Bacharelado em Informática,Bacharelado em Informática,2000/1,Cancelada por abandono,M,2000/1 - MAA123 Algebra para Informatica - 009...,2000/1 - 3.3\n2000/2 - 4.7\n2001/1 - 0.7,2000/1 - 3.3\n2000/2 - 3.9\n2001/1 - 3.2,3,390,3.2,2001/2 | 2002/1,NaN,2001/1,2000/1 - MAA123 Algebra para Informatica - 009...
3,3,2000/1,Bacharelado em Informática,Bacharelado em Ciência da Computação,2000/1,Cancelada por conclusão de curso,M,2000/1 - MAA123 Algebra para Informatica - 040...,2000/1 - 6.3\n2000/2 - 5.1\n2001/1 - 2.8\n2001...,2000/1 - 6.3\n2000/2 - 5.7\n2001/1 - 4.7\n2001...,17,3105,5.1,NaN,NaN,2001/1 | 2004/2,2000/1 - MAA123 Algebra para Informatica - 040...
4,4,2000/1,Bacharelado em Informática,Bacharelado em Ciência da Computação,2000/1,Cancelada por conclusão de curso,M,2000/1 - MAA123 Algebra para Informatica - 081...,2000/1 - 8.3\n2000/2 - 7\n2001/1 - 7.2\n2001/2...,2000/1 - 8.4\n2000/2 - 7.7\n2001/1 - 7.6\n2001...,10,3225,8.1,NaN,2004/2,NaN,NaN


In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2817 entries, 0 to 2816
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   matriculaDRE               2817 non-null   int64  
 1   periodoIngressoUFRJ        2817 non-null   object 
 2   cursoIngressoUFRJ          2817 non-null   object 
 3   cursoAtual                 2817 non-null   object 
 4   periodoIngressoCursoAtual  2817 non-null   object 
 5   situacaoMatriculaAtual     2817 non-null   object 
 6   sexo                       2817 non-null   object 
 7   disciplinasCursadas        2650 non-null   object 
 8   crPorPeriodo               2644 non-null   object 
 9   craPorPeriodo              2642 non-null   object 
 10  periodosIntegralizados     2817 non-null   int64  
 11  cargaHorariaAcumulada      2817 non-null   int64  
 12  CRA                        2654 non-null   float64
 13  periodosTrancados          948 non-null    objec

In [129]:
for coluna in df.columns:
    print(coluna)

matriculaDRE
periodoIngressoUFRJ
cursoIngressoUFRJ
cursoAtual
periodoIngressoCursoAtual
situacaoMatriculaAtual
sexo
disciplinasCursadas
crPorPeriodo
craPorPeriodo
periodosIntegralizados
cargaHorariaAcumulada
CRA
periodosTrancados
periodosCancelados
periodosCRMenor3
reprovacoes


In [130]:
df.iloc[2000:2004].T

,2000,2001,2002,2003
matriculaDRE,2000,2001,2002,2003
periodoIngressoUFRJ,2015/2,2015/2,2015/2,2015/2
cursoIngressoUFRJ,Engenharia Elétrica,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação
cursoAtual,Ciência da Computação,Ciência da Computação,Ciência da Computação,Ciência da Computação
periodoIngressoCursoAtual,2021/1,2021/1,2021/1,2021/1
situacaoMatriculaAtual,Ativa,Rematrícula por destrancamento ou descancelamento,Ativa,Trancamento automático: Perigo de cancelamento...
sexo,M,M,M,M
disciplinasCursadas,2015/0 - EET200 Int a Eng Metalurg e de Mater ...,2015/0 - CMA110 Tóp Gerais Ciências da Terra -...,2015/0 - CMT111 Inovações Cient Tecn e Art I -...,2015/2 - MAB111 Fund da Computação Digital - 0...
crPorPeriodo,2015/2 - 8.7\n2016/1 - 9.4\n2016/2 - 7.9\n2017...,2015/2 - 5.3\n2018/1 - 8.7\n2018/2 - 7.1\n2019...,2015/2 - 6.5\n2016/1 - 7.1\n2017/1 - 3.2\n2017...,2015/2 - 0.6\n2016/1 - 1.9\n2016/2 - 6\n2017/1...
craPorPeriodo,2015/2 - 8.5\n2016/1 - 8.8\n2016/2 - 8.6\n2017...,2015/2 - 7.1\n2018/1 - 7.7\n2018/2 - 7.6\n2019...,2015/2 - 6.8\n2016/1 - 6.9\n2017/1 - 6\n2017/2...,2015/2 - 0.7\n2016/1 - 1.2\n2016/2 - 1.6\n2017...


In [131]:
df.dropna(subset=['disciplinasCursadas'], inplace=True)

In [132]:
df['disciplinasCursadas'] = df['disciplinasCursadas'].astype(str)

## Disciplinas

In [133]:
df['disciplinasCursadas'] = df['disciplinasCursadas'].apply(lambda x: [tuple(item.split('-')) for item in x.split('\n')])

In [134]:
df['disciplinasCursadas'][0][0]

('2000/1 ', ' MAA123 Algebra para Informatica ', ' 060 ', ' Aprovado        ')

In [135]:
df_exploded = df.explode('disciplinasCursadas')

In [136]:
df_exploded.iloc[-3:-1].T

,2779,2779
matriculaDRE,2779,2779
periodoIngressoUFRJ,2022/1,2022/1
cursoIngressoUFRJ,Ciência da Computação,Ciência da Computação
cursoAtual,Ciência da Computação,Ciência da Computação
periodoIngressoCursoAtual,2022/1,2022/1
situacaoMatriculaAtual,Ativa,Ativa
sexo,M,M
disciplinasCursadas,"(2022/1 , ICP120 Computação I (CC) , 075 , ...","(2022/1 , ICP136 Introd Pensamento Dedutivo ,..."
crPorPeriodo,2022/1 - 7.8,2022/1 - 7.8
craPorPeriodo,2022/1 - 7.8,2022/1 - 7.8


In [137]:
df_exploded['disciplinasCursadas'] = df_exploded['disciplinasCursadas'].apply(lambda x: (
    x[0].replace(' ', ''),
    x[1].split()[0].replace(' ', ''),
    ' '.join(x[1].split()[1:]),
    x[2].replace(' ', ''),
    x[3].replace(' ', '')
))

In [139]:
df_exploded.iloc[-1]['disciplinasCursadas']

('2022/1', 'ICP624', 'Números Inteiros Criptografia', '074', 'Aprovado')

In [144]:
d_disciplina = pd.DataFrame(df_exploded['disciplinasCursadas'].tolist(), index=df_exploded.index, columns=('periodo','cod_Disciplina','disciplina','grau_disciplina','situacao_disciplina'))
d_disciplina

,periodo,cod_Disciplina,disciplina,grau_disciplina,situacao_disciplina
0,2000/1,MAA123,Algebra para Informatica,060,Aprovado
0,2000/1,MAB111,Fund da Computação Digital,089,Aprovado
0,2000/1,MAB120,Computacao para Informatica,094,Aprovado
0,2000/1,MAE111,Cálculo Infinitesimal I,056,Aprovado
0,2000/1,MAE115,Cálculo Vetorial e G Analitica,085,Aprovado
...,...,...,...,...,...
2779,2022/1,ICP111,Fund Computação Digital,067,Aprovado
2779,2022/1,ICP112,Sistemas de Informação,082,Aprovado
2779,2022/1,ICP120,Computação I (CC),075,Aprovado
2779,2022/1,ICP136,Introd Pensamento Dedutivo,081,Aprovado


In [147]:
df = pd.concat([df_exploded, d_disciplina], axis=1)
df.drop('disciplinasCursadas', axis=1, inplace=True)
df

,matriculaDRE,periodoIngressoUFRJ,cursoIngressoUFRJ,cursoAtual,periodoIngressoCursoAtual,situacaoMatriculaAtual,sexo,crPorPeriodo,craPorPeriodo,periodosIntegralizados,...,CRA,periodosTrancados,periodosCancelados,periodosCRMenor3,reprovacoes,periodo,cod_Disciplina,disciplina,grau_disciplina,situacao_disciplina
0,0,2000/1,Bacharelado em Informática,Bacharelado em Ciência da Computação,2000/1,Cancelada por conclusão de curso,M,2000/1 - 7.6\n2000/2 - 7.1\n2001/1 - 5.9\n2001...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,13,...,6.6,NaN,NaN,2005/2,2001/1 - MAB243 Organizacao de Dados I - 020 -...,2000/1,MAA123,Algebra para Informatica,060,Aprovado
0,0,2000/1,Bacharelado em Informática,Bacharelado em Ciência da Computação,2000/1,Cancelada por conclusão de curso,M,2000/1 - 7.6\n2000/2 - 7.1\n2001/1 - 5.9\n2001...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,13,...,6.6,NaN,NaN,2005/2,2001/1 - MAB243 Organizacao de Dados I - 020 -...,2000/1,MAB111,Fund da Computação Digital,089,Aprovado
0,0,2000/1,Bacharelado em Informática,Bacharelado em Ciência da Computação,2000/1,Cancelada por conclusão de curso,M,2000/1 - 7.6\n2000/2 - 7.1\n2001/1 - 5.9\n2001...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,13,...,6.6,NaN,NaN,2005/2,2001/1 - MAB243 Organizacao de Dados I - 020 -...,2000/1,MAB120,Computacao para Informatica,094,Aprovado
0,0,2000/1,Bacharelado em Informática,Bacharelado em Ciência da Computação,2000/1,Cancelada por conclusão de curso,M,2000/1 - 7.6\n2000/2 - 7.1\n2001/1 - 5.9\n2001...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,13,...,6.6,NaN,NaN,2005/2,2001/1 - MAB243 Organizacao de Dados I - 020 -...,2000/1,MAE111,Cálculo Infinitesimal I,056,Aprovado
0,0,2000/1,Bacharelado em Informática,Bacharelado em Ciência da Computação,2000/1,Cancelada por conclusão de curso,M,2000/1 - 7.6\n2000/2 - 7.1\n2001/1 - 5.9\n2001...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,13,...,6.6,NaN,NaN,2005/2,2001/1 - MAB243 Organizacao de Dados I - 020 -...,2000/1,MAE115,Cálculo Vetorial e G Analitica,085,Aprovado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2779,2779,2022/1,Ciência da Computação,Ciência da Computação,2022/1,Ativa,M,2022/1 - 7.8,2022/1 - 7.8,1,...,7.8,NaN,NaN,NaN,NaN,2022/1,ICP111,Fund Computação Digital,067,Aprovado
2779,2779,2022/1,Ciência da Computação,Ciência da Computação,2022/1,Ativa,M,2022/1 - 7.8,2022/1 - 7.8,1,...,7.8,NaN,NaN,NaN,NaN,2022/1,ICP112,Sistemas de Informação,082,Aprovado
2779,2779,2022/1,Ciência da Computação,Ciência da Computação,2022/1,Ativa,M,2022/1 - 7.8,2022/1 - 7.8,1,...,7.8,NaN,NaN,NaN,NaN,2022/1,ICP120,Computação I (CC),075,Aprovado
2779,2779,2022/1,Ciência da Computação,Ciência da Computação,2022/1,Ativa,M,2022/1 - 7.8,2022/1 - 7.8,1,...,7.8,NaN,NaN,NaN,NaN,2022/1,ICP136,Introd Pensamento Dedutivo,081,Aprovado


In [150]:
df.iloc[1].T

matriculaDRE                                                                 0
periodoIngressoUFRJ                                                     2000/1
cursoIngressoUFRJ                                   Bacharelado em Informática
cursoAtual                                Bacharelado em Ciência da Computação
periodoIngressoCursoAtual                                               2000/1
situacaoMatriculaAtual                        Cancelada por conclusão de curso
sexo                                                                         M
crPorPeriodo                 2000/1 - 7.6\n2000/2 - 7.1\n2001/1 - 5.9\n2001...
craPorPeriodo                2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...
periodosIntegralizados                                                      13
cargaHorariaAcumulada                                                     3105
CRA                                                                        6.6
periodosTrancados                                   

In [149]:
df.iloc[1]['crPorPeriodo']

'2000/1 - 7.6\n2000/2 - 7.1\n2001/1 - 5.9\n2001/2 - 6.8\n2002/1 - 6.3\n2002/2 - 5.6\n2003/1 - 7.9\n2003/2 - 6.7\n2004/1 - 7.8\n2004/2 - 6.2\n2005/1 - 6.3\n2005/2 - 1.8\n2006/1 - 6'

## Periodo e CR

In [ ]:
df['crPorPeriodo'] = df['crPorPeriodo'].astype(str)

In [178]:
df['crPorPeriodo'] = df['crPorPeriodo'].apply(lambda x: [tuple(item.split(' - ')) for item in x.split('\n')])
df_exploded = df.explode('crPorPeriodo')

In [167]:
df_exploded[['PERIODO', 'CR']] = pd.DataFrame(df_exploded['crPorPeriodo'].tolist(), index=df_exploded.index)
df_exploded.drop(columns=['crPorPeriodo'], inplace=True)
df_exploded.reset_index(drop=True, inplace=True)

df_exploded

,matriculaDRE,periodoIngressoUFRJ,cursoIngressoUFRJ,cursoAtual,periodoIngressoCursoAtual,situacaoMatriculaAtual,sexo,disciplinasCursadas,craPorPeriodo,periodosIntegralizados,cargaHorariaAcumulada,CRA,periodosTrancados,periodosCancelados,periodosCRMenor3,reprovacoes,PERIODO,CR
0,0,2000/1,Bacharelado em Informática,Bacharelado em Ciência da Computação,2000/1,Cancelada por conclusão de curso,M,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,13,3105,6.6,NaN,NaN,2005/2,2001/1 - MAB243 Organizacao de Dados I - 020 -...,2000/1,7.6
1,0,2000/1,Bacharelado em Informática,Bacharelado em Ciência da Computação,2000/1,Cancelada por conclusão de curso,M,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,13,3105,6.6,NaN,NaN,2005/2,2001/1 - MAB243 Organizacao de Dados I - 020 -...,2000/2,7.1
2,0,2000/1,Bacharelado em Informática,Bacharelado em Ciência da Computação,2000/1,Cancelada por conclusão de curso,M,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,13,3105,6.6,NaN,NaN,2005/2,2001/1 - MAB243 Organizacao de Dados I - 020 -...,2001/1,5.9
3,0,2000/1,Bacharelado em Informática,Bacharelado em Ciência da Computação,2000/1,Cancelada por conclusão de curso,M,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,13,3105,6.6,NaN,NaN,2005/2,2001/1 - MAB243 Organizacao de Dados I - 020 -...,2001/2,6.8
4,0,2000/1,Bacharelado em Informática,Bacharelado em Ciência da Computação,2000/1,Cancelada por conclusão de curso,M,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,13,3105,6.6,NaN,NaN,2005/2,2001/1 - MAB243 Organizacao de Dados I - 020 -...,2002/1,6.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22877,2812,2022/2,Ciência da Computação,Ciência da Computação,2022/2,Ativa,M,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,nan,None
22878,2813,2022/2,Ciência da Computação,Ciência da Computação,2022/2,Ativa,F,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,nan,None
22879,2814,2022/2,Ciência da Computação,Ciência da Computação,2022/2,Ativa,M,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,nan,None
22880,2815,2022/2,Ciência da Computação,Ciência da Computação,2022/2,Ativa,F,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,nan,None


,matriculaDRE,periodoIngressoUFRJ,cursoIngressoUFRJ,cursoAtual,periodoIngressoCursoAtual,situacaoMatriculaAtual,sexo,disciplinasCursadas,craPorPeriodo,periodosIntegralizados,cargaHorariaAcumulada,CRA,periodosTrancados,periodosCancelados,periodosCRMenor3,reprovacoes,PERIODO,CR
0,0,2000/1,Bacharelado em Informática,Bacharelado em Ciência da Computação,2000/1,Cancelada por conclusão de curso,M,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,13,3105,6.6,NaN,NaN,2005/2,2001/1 - MAB243 Organizacao de Dados I - 020 -...,2000/1,7.6
1,0,2000/1,Bacharelado em Informática,Bacharelado em Ciência da Computação,2000/1,Cancelada por conclusão de curso,M,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,13,3105,6.6,NaN,NaN,2005/2,2001/1 - MAB243 Organizacao de Dados I - 020 -...,2000/2,7.1
2,0,2000/1,Bacharelado em Informática,Bacharelado em Ciência da Computação,2000/1,Cancelada por conclusão de curso,M,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,13,3105,6.6,NaN,NaN,2005/2,2001/1 - MAB243 Organizacao de Dados I - 020 -...,2001/1,5.9
3,0,2000/1,Bacharelado em Informática,Bacharelado em Ciência da Computação,2000/1,Cancelada por conclusão de curso,M,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,13,3105,6.6,NaN,NaN,2005/2,2001/1 - MAB243 Organizacao de Dados I - 020 -...,2001/2,6.8
4,0,2000/1,Bacharelado em Informática,Bacharelado em Ciência da Computação,2000/1,Cancelada por conclusão de curso,M,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,13,3105,6.6,NaN,NaN,2005/2,2001/1 - MAB243 Organizacao de Dados I - 020 -...,2002/1,6.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22877,2812,2022/2,Ciência da Computação,Ciência da Computação,2022/2,Ativa,M,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,nan,None
22878,2813,2022/2,Ciência da Computação,Ciência da Computação,2022/2,Ativa,F,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,nan,None
22879,2814,2022/2,Ciência da Computação,Ciência da Computação,2022/2,Ativa,M,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,nan,None
22880,2815,2022/2,Ciência da Computação,Ciência da Computação,2022/2,Ativa,F,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,nan,None


In [108]:
df['crPorPeriodo'].apply(lambda row: str(row).split('\n')).apply(lambda row: str(row).split(' - '))[0]

["['2000/1",
 "7.6', '2000/2",
 "7.1', '2001/1",
 "5.9', '2001/2",
 "6.8', '2002/1",
 "6.3', '2002/2",
 "5.6', '2003/1",
 "7.9', '2003/2",
 "6.7', '2004/1",
 "7.8', '2004/2",
 "6.2', '2005/1",
 "6.3', '2005/2",
 "1.8', '2006/1",
 "6']"]